In [1]:
import spacy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
root_dir = ('/content/drive/MyDrive/fake_and_real_news')
os.chdir(root_dir)

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/drive/MyDrive/fake_and_real_news/fake_or_real_news.csv')

In [6]:
print(df.head(5))

   Unnamed: 0                                              title  \
0        8476                       You Can Smell Hillary’s Fear   
1       10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2        3608        Kerry to go to Paris in gesture of sympathy   
3       10142  Bernie supporters on Twitter erupt in anger ag...   
4         875   The Battle of New York: Why This Primary Matters   

                                                text label  
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE  
2  U.S. Secretary of State John F. Kerry said Mon...  REAL  
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE  
4  It's primary day in New York and front-runners...  REAL  


In [7]:
df1 = df.drop(['Unnamed: 0' , 'title'] , axis = 1)

In [8]:
print(df1.head(5))

                                                text label
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE
2  U.S. Secretary of State John F. Kerry said Mon...  REAL
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE
4  It's primary day in New York and front-runners...  REAL


In [9]:
print(df1.tail())

                                                   text label
6330  The State Department told the Republican Natio...  REAL
6331  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...  FAKE
6332   Anti-Trump Protesters Are Tools of the Oligar...  FAKE
6333  ADDIS ABABA, Ethiopia —President Obama convene...  REAL
6334  Jeb Bush Is Suddenly Attacking Trump. Here's W...  REAL


In [10]:
print(df1.shape)

(6335, 2)


In [11]:
### check the distribution of data

## check the imbalance dataset

In [12]:
df1['label'].value_counts()

,count
label,
REAL,3171
FAKE,3164


### change the categorical data to binary data  0 / 1

In [13]:
df1['label_num'] = df['label'].map(
    {
        'FAKE' : 0,
        'REAL' : 1
    }
)

In [14]:
df1.head()

,text,label,label_num
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,0
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,0
2,U.S. Secretary of State John F. Kerry said Mon...,REAL,1
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,0
4,It's primary day in New York and front-runners...,REAL,1


In [15]:
import spacy

In [16]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [17]:
nlp = spacy.load('en_core_web_lg')

### convert our text into vetor using above library

In [18]:
## convert text to vector


"""for i in range(len(df1)):
     df1['vector'][i] = nlp(df1['text'][i]).vector"""

## using lambda function

df1['vector']  = df1['text'].apply(lambda text : nlp(text).vector)

In [19]:
 len(df1)

6335

In [21]:
print(df1.head())     ## for invidual text - vector form is 300

                                                text label  label_num  \
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE          0   
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE          0   
2  U.S. Secretary of State John F. Kerry said Mon...  REAL          1   
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE          0   
4  It's primary day in New York and front-runners...  REAL          1   

                                              vector  
0  [-1.3751823, 1.3421849, -2.3666484, 0.12908486...  
1  [-1.7449774, 0.93961924, -2.024867, 0.42536643...  
2  [-1.9426425, 1.0062195, -1.9992222, 0.20469022...  
3  [-1.9125352, -0.1481846, -1.1432766, 0.6861217...  
4  [-1.8516092, 1.3163909, -2.1726575, 1.2286776,...  


In [52]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train   , y_test = train_test_split(
    df1.vector.values,
    df1.label_num,
    test_size = 0.2,
    random_state = 42
)

In [53]:
X_train.shape

(5068,)

In [54]:
X_test.shape

(1267,)

In [55]:
import numpy as np

In [56]:
## vector ke uper hum stack karege
## now currently is X_train is in vector form

X_train_stack = np.stack(X_train)
X_test_stack = np.stack(X_test)

In [57]:
print(X_train_stack.shape)
print(X_test_stack.shape)

(5068, 300)
(1267, 300)


### TRain the model

In [58]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()

model.fit(X_train_stack , y_train)



ValueError: Negative values in data passed to MultinomialNB (input X)

### Negative values in data passed to MultinomialNB (input X)

aisa error throw kare tho hame model ko scale karna padega

In [59]:

from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_stack_sc  = scaler.fit_transform(X_train_stack)
X_test_stack_sc  = scaler.transform(X_test_stack)

model.fit(X_train_stack_sc , y_train)



MultinomialNB()

In [60]:
y_pred = model.predict(X_test_stack_sc)

In [61]:
y_pred

array([0, 1, 0, ..., 0, 0, 1])

In [62]:
y_test

,label_num
1357,0
2080,0
2718,0
812,0
4886,0
...,...
1512,0
57,1
6092,1
3403,1


In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test , y_pred)

0.7640094711917916

In [64]:
from sklearn.metrics import classification_report
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.81      0.68      0.74       628
           1       0.73      0.84      0.78       639

    accuracy                           0.76      1267
   macro avg       0.77      0.76      0.76      1267
weighted avg       0.77      0.76      0.76      1267



In [65]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test , y_pred))

[[429 199]
 [100 539]]


In [66]:
model = MultinomialNB(alpha = 0.1)
model.fit(X_train_stack_sc , y_train)

MultinomialNB(alpha=0.1)

In [67]:
y_pred = model.predict(X_test_stack_sc)
y_pred

array([0, 1, 0, ..., 0, 0, 1])

In [68]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test , y_pred)

0.7647987371744278

In [101]:
"""import numpy as np


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_stack_sc  = scaler.fit_transform(X_train_stack)
X_test_stack_sc  = scaler.transform(X_test_stack)

previous_score =0

for alpha in np.arange(0,1,0.1):                    ### 0.1 is step
  model = MultinomialNB(alpha = alpha)

  model.fit(X_train_stack_sc , y_train)
  model.predict(X_test_stack_sc)
  sc = accuracy_score(y_test , y_pred)
  if sc>previous_score:
    model_new = model"""

'import numpy as np\n\n\nfrom sklearn.preprocessing import MinMaxScaler\nscaler = MinMaxScaler()\n\nX_train_stack_sc  = scaler.fit_transform(X_train_stack)\nX_test_stack_sc  = scaler.transform(X_test_stack)\n\nprevious_score =0\n\nfor alpha in np.arange(0,1,0.1):                    ### 0.1 is step\n  model = MultinomialNB(alpha = alpha)\n\n  model.fit(X_train_stack_sc , y_train)\n  model.predict(X_test_stack_sc)\n  sc = accuracy_score(y_test , y_pred)\n  if sc>previous_score:\n    model_new = model'

In [74]:
y_pred = model_new.predict(X_test_stack_sc)

In [75]:
y_pred

array([0, 1, 0, ..., 0, 0, 1])

In [76]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test , y_pred)

0.7632202052091555

### Gensim library - conduct word2vec